# Analysis - exp13 and 14

Repeats of stumbler (13) and stumbler-strategist (14) exps, with optimal parameters.

In [ ]:
import os
import csv
import numpy as np
#import torch as th

from glob import glob
from pprint import pprint

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from notebook_helpers import load_monitored
from notebook_helpers import join_monitored
from notebook_helpers import score_summary

# Find the files
## Exp 13

In [ ]:
path = "/Users/Jack/Desktop/PNC/CoAx/azad-next-move/data/wythoff/exp13/"
#results_path = "/Users/type/Code/azad/data/wythoff/"

files = sorted(glob(os.path.join(path, "run_*_monitor.csv")))
files[:10]

exp_13 = []
for fi in files:
    mon = load_monitored(fi)
    exp_13.append(mon)

## Exp 14

In [ ]:
path = "/Users/Jack/Desktop/PNC/CoAx/azad-next-move/data/wythoff/exp14/"

exp_14 = []
runs = range(1, 3)
for r in runs:
    files = sorted(glob(os.path.join(path, 
        "run_{}_episode*_stumbler*.csv".format(int(r)))))
    
    mon = join_monitored(files, sort_key='episode')
    exp_14.append(mon)

In [ ]:
path = "/Users/Jack/Desktop/PNC/CoAx/azad-next-move/data/wythoff/exp14_replay/"

exp_14_replay = []
runs = range(1, 101)
for r in runs:
    files = sorted(glob(os.path.join(path, 
        "run_{}_episode*_stumbler*.csv".format(int(r)))))
    
    mon = join_monitored(files, sort_key='episode')
    exp_14_replay.append(mon)

In [ ]:
pprint(exp_13[0].keys())
pprint(exp_14[0].keys())
pprint(exp_14_replay[0].keys())

In [ ]:
print(np.max([np.max(mon['score'][-100:-1]) for mon in exp_13]))
print(np.max([np.max(mon['score'][-100:-1]) for mon in exp_14]))
print(np.max([np.max(mon['score'][-100:-1]) for mon in exp_14_replay]))

# Plots

## Optimal play

In [ ]:
plt.figure(figsize=(5, 2.5))

for n, mon in enumerate(exp_13):
    # Only want one legend entry
    lab = None
    if n == 0:
        lab = "Stumbler"
        
    # Plot
    _ = plt.plot(mon['episode'], mon['score'], 
                    color='grey', alpha=.1,  label=lab)

for n, mon in enumerate(exp_14):
    # Only want one legend entry
    lab = None
    if n == 0:
        lab = "Strategist"
    
    # Plot
    _ = plt.plot(mon['episode'], mon['score'], 
                    color='red', alpha=.1, label=lab)

_ = plt.tight_layout()
_ = plt.legend()
_ = plt.xlabel("Episode")
_ = plt.ylabel("Optimal play score")

In [ ]:
episode_13,        exp_13_avg,        exp_13_SE        = score_summary(exp_13)
episode_14,        exp_14_avg,        exp_14_SE        = score_summary(exp_14)
episode_14_replay, exp_14_replay_avg, exp_14_replay_SE = score_summary(exp_14_replay)

In [ ]:
plt.figure(figsize=(7, 2.5))
_ = plt.plot(episode_13, exp_13_avg, linestyle="--", color='grey', alpha=.5, linewidth=3, label="Stumbler alone")
_ = plt.plot(episode_14, exp_14_avg, color='red', alpha=.5, linewidth=3, label="with Strategist and imagination")
_ = plt.plot(episode_14_replay, exp_14_replay_avg, color='blue', alpha=.5, linewidth=3, label="with Strategist and replay")
_ = plt.tight_layout()
_ = plt.legend()
_ = plt.xlabel("Episode")
_ = plt.ylabel("Optimal play score")

In [ ]:
plt.figure()

plt.bar(range(3), [exp_14_avg[-1], exp_14_replay_avg[-1], exp_13_avg[-1]],
             yerr=[exp_14_SE[-1],  exp_14_replay_SE[-1],  exp_13_SE[-1] ], capsize=5)

_ = plt.tight_layout()
# _ = plt.ylim(0, 1)
_ = plt.xlabel("Agent")
_ = plt.ylabel("Ending optimal move proportion")
_ = plt.xticks(range(3), labels = ["stumbler-strategist", "stumbler-recaller", "stumbler alone"])

In [ ]:
plt.figure()

plt.bar(range(2), [exp_14_avg[-1], exp_14_replay_avg[-1]],
             yerr=[exp_14_SE[-1],  exp_14_replay_SE[-1] ], capsize=5)

_ = plt.tight_layout()
_ = plt.ylim(0.72, 0.79)
_ = plt.xlabel("Agent")
_ = plt.ylabel("Ending optimal move proportion")
_ = plt.xticks(range(2), labels = ["stumbler-strategist", "stumbler-recaller"])

## Total reward

In [ ]:
plt.figure(figsize=(5, 2.5))

for n, mon in enumerate(exp_13):
    # Only want one legend entry
    lab = None
    if n == 0:
        lab = "Stumbler"
        
    # Plot
    _ = plt.plot(
            mon['episode'], 
            np.asarray(mon['total_reward']),
            color='grey', alpha=1,  label=lab
    )

for n, mon in enumerate(exp_14):
    # Only want one legend entry
    lab = None
    if n == 0:
        lab = "Strategist"
    
    # Plot
    _ = plt.plot(
            mon['episode'], 
            np.asarray(mon['total_reward']),
            color='red', alpha=1, label=lab
    )

_ = plt.tight_layout()
_ = plt.legend()
_ = plt.xlabel("Episode")
_ = plt.ylabel("Total reward")

In [ ]:
# def load_stumbler_data(path, index_range=(0, 20)):
#     exps = []
#     runs = range(index_range[0], index_range[1])
#     for r in runs:
#         files = sorted(glob(os.path.join(path, 
#             "run_{}_episode*_stumbler*".format(int(r)))))

#         mon = join_monitored(files, sort_key='episode')
#         exps.append(mon)
#     return exps

In [ ]:
# path = "/Users/Jack/Desktop/PNC/CoAx/azad-next-move/data/wythoff/exp14/"
# exp_14 = load_stumbler_data(path, (1, 21))

# path = "/Users/Jack/Desktop/PNC/CoAx/azad-next-move/data/wythoff/exp14_replay/"
# exp_14_replay = load_stumbler_data(path, (1, 21))

# path = "/Users/Jack/Desktop/PNC/CoAx/azad-next-move/data/wythoff/exp13/"
# exp_13 = load_stumbler_data(path, (1, 21))

In [ ]:
pprint(exp_13[0].keys())
pprint(exp_14[0].keys())
pprint(exp_14_replay[0].keys())

In [ ]:
# print(exp_14[0]['total_wins'][475:525])
# # print(exp_14[0]['total_wins'][100])

# for i in range(len(exp_14)):
#     exp_14[i]['total_wins'] = np.array(exp_14[i]['total_wins'])
#     add = 0
#     for j in range(int(len(exp_14[i]['total_wins']) / 100)):
#         exp_14[i]['total_wins'][j * 100 : (j + 1) * 100] += add
#         add = exp_14[i]['total_wins'][((j + 1) * 100) - 1]

# print(exp_14[0]['total_wins'][475:525])
# # print(exp_14[0]['total_wins'][100])

# for i in range(len(exp_14_replay)):
#     exp_14_replay[i]['total_wins'] = np.array(exp_14_replay[i]['total_wins'])
#     add = 0
#     for j in range(int(len(exp_14_replay[i]['total_wins']) / 100)):
#         exp_14_replay[i]['total_wins'][j * 100 : (j + 1) * 100] += add
#         add = exp_14_replay[i]['total_wins'][((j + 1) * 100) - 1]

        

# plt.figure(figsize=(5, 2.5))

# # for n, mon in enumerate(exp_13):
# #     # Only want one legend entry
# #     lab = None
# #     if n == 0:
# #         lab = "Stumbler"
        
# #     # Plot
# #     _ = plt.plot(
# #             mon['episode'], 
# #             np.asarray(mon['total_wins']),
# #             color='grey', alpha=1,  label=lab
# #     )

# # for n, mon in enumerate(exp_14):
# #     # Only want one legend entry
# #     lab = None
# #     if n == 0:
# #         lab = "Strategist"
    
# #     # Plot
# #     _ = plt.plot(
# #             mon['episode'], 
# #             np.asarray(mon['total_wins']),
# #             color='red', alpha=1, label=lab
# #     )

# # _ = plt.tight_layout()
# # _ = plt.legend()
# # _ = plt.xlabel("Episode")
# # _ = plt.ylabel("Total wins")
# # _ = plt.xlim(400,600)
# # _ = plt.ylim(0,1000)

In [ ]:
episode_13,        exp_13_avg,        exp_13_SE        = score_summary(exp_13,        key="total_reward")
episode_14,        exp_14_avg,        exp_14_SE        = score_summary(exp_14,        key="total_reward")
episode_14_replay, exp_14_replay_avg, exp_14_replay_SE = score_summary(exp_14_replay, key="total_reward")

In [ ]:
plt.figure()

plt.bar(range(3), [exp_14_avg[-1], exp_14_replay_avg[-1], exp_13_avg[-1]],
             yerr=[exp_14_SE[-1],  exp_14_replay_SE[-1],  exp_13_SE[-1] ], capsize=5)

_ = plt.tight_layout()
# _ = plt.ylim(0, 1)
_ = plt.xlabel("Agent")
_ = plt.ylabel("Cumulative reward")
_ = plt.xticks(range(3), labels = ["stumbler-strategist", "stumbler-recaller", "stumbler"])

In [ ]:
plt.figure()

plt.bar(range(3), [exp_14_avg[-1], exp_14_replay_avg[-1], exp_13_avg[-1]],
             yerr=[exp_14_SE[-1],  exp_14_replay_SE[-1],  exp_13_SE[-1] ], capsize=5)

_ = plt.tight_layout()
_ = plt.ylim(68000, 71000)
_ = plt.xlabel("Agent")
_ = plt.ylabel("Cumulative reward")
_ = plt.xticks(range(3), labels = ["stumbler-strategist", "stumbler-recaller", "stumbler"])

In [ ]:
plt.figure()

plt.bar(range(3), [exp_14_avg[-1]        / max(episode_14),
                   exp_14_replay_avg[-1] / max(episode_14_replay),
                   exp_13_avg[-1]        / max(episode_13)],
        yerr=[exp_14_SE[-1]        / max(episode_14),
              exp_14_replay_SE[-1] / max(episode_14_replay),
              exp_13_SE[-1]        / max(episode_13)],
        capsize=5)

_ = plt.tight_layout()
# _ = plt.ylim(0, 1)
_ = plt.xlabel("Agent")
_ = plt.ylabel("Average reward per game")
_ = plt.xticks(range(3), labels = ["stumbler-strategist", "stumbler-recaller", "stumbler"])

In [ ]:
plt.figure()

plt.bar(range(3), [exp_14_avg[-1]        / max(episode_14),
                   exp_14_replay_avg[-1] / max(episode_14_replay),
                   exp_13_avg[-1]        / max(episode_13)],
        yerr=[exp_14_SE[-1]        / max(episode_14),
              exp_14_replay_SE[-1] / max(episode_14_replay),
              exp_13_SE[-1]        / max(episode_13)],
        capsize=5)

_ = plt.tight_layout()
_ = plt.ylim(0.9, 0.96)
_ = plt.xlabel("Agent")
_ = plt.ylabel("Average reward per game")
_ = plt.xticks(range(3), labels = ["stumbler-strategist", "stumbler-recaller", "stumbler"])

In [ ]:
episode_13,        exp_13_avg,        exp_13_SE        = score_summary(exp_13,        key="total_wins")
episode_14,        exp_14_avg,        exp_14_SE        = score_summary(exp_14,        key="total_wins")
episode_14_replay, exp_14_replay_avg, exp_14_replay_SE = score_summary(exp_14_replay, key="total_wins")

In [ ]:
plt.figure()

plt.bar(range(3), [exp_14_avg[-1], exp_14_replay_avg[-1], exp_13_avg[-1]],
             yerr=[exp_14_SE[-1],  exp_14_replay_SE[-1],  exp_13_SE[-1] ], capsize=5)

_ = plt.tight_layout()
# _ = plt.ylim(0, 1)
_ = plt.xlabel("Agent")
_ = plt.ylabel("Cumulative games won")
_ = plt.xticks(range(3), labels = ["stumbler-strategist", "stumbler-recaller", "stumbler"])

In [ ]:
plt.figure()
plt.plot(episode_14, exp_14_avg)

_ = plt.tight_layout()
# _ = plt.legend()
_ = plt.xlabel("Episode")
_ = plt.ylabel("Total wins")
# _ = plt.xlim(400,600)
# _ = plt.ylim(0,1000)

In [ ]:
path = "/Users/Jack/Desktop/PNC/CoAx/azad-next-move/data/wythoff/exp14/"

exp_14 = []
runs = range(1, 21)
for r in runs:
    files = sorted(glob(os.path.join(path, 
        "run_{}_episode*_strategist*.csv".format(int(r)))))
    
    mon = join_monitored(files, sort_key='episode')
    exp_14.append(mon)

In [ ]:
pprint(exp_14[0].keys())

In [ ]:
plt.figure(figsize=(5, 2.5))

for n, mon in enumerate(exp_14):
    # Only want one legend entry
    lab = None
    if n == 0:
        lab = "Strategist"
    
    # Plot
    _ = plt.plot(mon['episode'], mon['mae_euclid'], 
                    color='red', alpha=.1, label=lab)

_ = plt.tight_layout()
_ = plt.legend()
_ = plt.xlabel("Episode")
_ = plt.ylabel("mae_euclid")

In [ ]:
episode_14_euc, exp_14_avg_euc, _ = score_summary(exp_14, key='mae_euclid')

In [ ]:
plt.figure(figsize=(7, 2.5))
_ = plt.plot(episode_14_euc, exp_14_avg_euc, color='red', alpha=.5, linewidth=3, label="Strategist")
_ = plt.tight_layout()
_ = plt.legend()
_ = plt.xlabel("Episode")
_ = plt.ylabel("mae_euclid")

In [ ]:
plt.figure(figsize=(5, 2.5))

for n, mon in enumerate(exp_14):
    # Only want one legend entry
    lab = None
    if n == 0:
        lab = "Strategist"
    
    # Plot
    _ = plt.plot(mon['episode'], mon['mae'], 
                    color='red', alpha=.1, label=lab)

_ = plt.tight_layout()
_ = plt.legend()
_ = plt.xlabel("Episode")
_ = plt.ylabel("mae")

In [ ]:
episode_14, exp_14_avg, _ = score_summary(exp_14, key='mae')

In [ ]:
plt.figure(figsize=(7, 2.5))
_ = plt.plot(episode_14, exp_14_avg, color='red', alpha=.5, linewidth=3, label="Strategist")
_ = plt.tight_layout()
_ = plt.legend()
_ = plt.xlabel("Episode")
_ = plt.ylabel("mae")

In [ ]:
plt.figure(figsize=(7, 2.5))

_ = plt.plot(episode_14_euc, exp_14_avg_euc, color='green', alpha=.5, linewidth=3, label="mae_euclid")
_ = plt.tight_layout()
_ = plt.legend()
_ = plt.xlabel("Episode")
# _ = plt.ylabel("mae_euclid")

_ = plt.plot(episode_14, exp_14_avg, color='purple', alpha=.5, linewidth=3, label="mae_wythoff's")
_ = plt.tight_layout()
_ = plt.legend()
_ = plt.xlabel("Episode")
_ = plt.ylabel("mae")

In [ ]:
max(episode_14)